In [1]:
from sklearn.datasets import fetch_kddcup99
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from time import time

In [2]:
#pip install scikit-learn

In [3]:
# import the python libraries to create/connect to a Spark Session
from pyspark.sql import SparkSession

# build a SparkSession 
#   connect to the master node on the port where the master node is listening (7077)
#   declare the app name 
#   configure the executor memory to 512 MB
#   either *connect* or *create* a new Spark Context
spark = SparkSession.builder \
    .master("spark://spark-master:7077")\
    .appName("My first spark application")\
    .config("spark.executor.memory", "512m")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/04 11:19:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# create a spark context
sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [5]:
nSlice = None # IMPORTANT PARAMETER FOR NUMBER OF PARTITIONS
Rdd = sc.parallelize(np.arange(10_000), numSlices = nSlice)

In [6]:
Rdd.getNumPartitions() # check partitions

2

In [7]:
#Rdd.collect()
Rdd.persist()

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289

In [8]:
def collatz(x):
    if x%2==0:
        return x/2
    else:
        return 3*x+1

In [9]:
%%time
for _ in range(200):
    Rdd=Rdd.map(collatz)
#Rdd.collect()

CPU times: user 2.55 ms, sys: 820 µs, total: 3.37 ms
Wall time: 3.79 ms


In [10]:
#%%time
Rdd=Rdd.persist()

In [11]:
'''
%%time
for _ in range(100):
    Rdd=Rdd#.persist()
    Rdd=Rdd.map(collatz)
    Rdd#.unpersist()
Rdd.collect();
'''

'\n%%time\nfor _ in range(100):\n    Rdd=Rdd#.persist()\n    Rdd=Rdd.map(collatz)\n    Rdd#.unpersist()\nRdd.collect();\n'

In [12]:
%%time
for _ in range(100):
    RddCached=Rdd.persist()
    RddCached=RddCached.map(collatz)
    RddCached.unpersist()
Rdd.collect();

CPU times: user 8.43 ms, sys: 3.16 ms, total: 11.6 ms
Wall time: 2.49 s
